# Load dataset

In [ ]:
import os
import pandas as pd
import numpy as np
dataset_path = ""
dataset_name = "dataset.csv"
dataset = pd.read_csv(os.path.join(dataset_path, dataset_name))
dataset.head()

In [ ]:
ids_all = dataset.id.value_counts()
ids = ids_all.index.values
print len(ids)
print dataset.shape
dataset = dataset[dataset.id.isin(ids)]
print dataset.shape

### split the dataset into before medication and after medication datasets

In [ ]:
dataset_before = dataset.iloc[::2]
dataset_after = dataset.iloc[1::2]

# 10-fold cross validation on patients

In [ ]:
LO = [1e7, 1e6, 1e5, 1e4, 1e3, 1e2, 1e1, 1, 1e-1, 1e-2] # play with range for lam0 and sweep to choose right penalty
LS = [1e7, 1e6, 1e5, 1e4, 1e3, 1e2, 1e1, 1, 1e-1, 1e-2] # play with range for lambda-S conditioned on lam0

#LO = [1e5] # play with range for lam0 and sweep to choose right penalty
#LS = [1e-2] # play with range for lambda-S conditioned on lam0
 
lo = []
ls = []
for i in LO:
    for j in LS:
        lo.append(i)
        ls.append(j)
print lo
print ls

In [ ]:
%reload_ext rpy2.ipython

In [ ]:
%%R 
setwd("/dssl")
source("dssFunctionLibrary.R")
source("dssDataSetSpecificHacks.R")
source("dssPerformanceEvaluation.R")
source("mimicUsefulFunction.R")
require(gridExtra)
require(ROCR)
require(ISLR)
require(caret)
require(magrittr)
library(foreach)
library(doMC)
registerDoMC(11)


In [ ]:
def train_model_in_R():
    %R -i XTrainScaled,XTrainTimeVector,lo,ls,sm_on_idx,sm_off_idx 
    %R OrderPairs <- data.frame(onIdx = seq(1, nrow(XTrainScaled)/2, by = 1), offIdx = seq(nrow(XTrainScaled)/2+1, nrow(XTrainScaled), by = 1))
    %R SmoothnessPairs <- data.frame(onIdx = sm_on_idx, offIdx = sm_off_idx)
    %R -o res res <- dssTrain.Linear(XTrainScaled, OrderPairs, SmoothnessPairs, XTrainTimeVector, lo, ls,"dssl_mdl_v1_tap2",doParallel = 1)
    return res

In [ ]:
def get_accuracy(scores):
    length = len(scores)/2
    diff = scores[:length] - scores[length:]
    accuracy = np.sum(diff >= 0) * 1.0 / len(diff)
    return accuracy

def evaluate(pairs, weights):
    # 0:17 voice
    # 17:143 balance 
    # 143:269 gait 
    # 269:420 tap 
    # 420:435 react
    scores = pairs.dot(weights)
    # 0:17 voice
    scores_voice = pairs[:, 0:17].dot(weights[0:17])
    # 17:143 balance
    scores_balance = pairs[:,17:143].dot(weights[17:143])
    
    # 143:269 gait
    scores_gait = pairs[:,143:269].dot(weights[143:269])
    
    # 269:420 tap
    scores_tap = pairs[:,269:420].dot(weights[269:420])
    
    # 420: react
    scores_react = pairs[:,420:].dot(weights[420: ])
    
    return {'scores':scores, 'accuracy':get_accuracy(scores), 
            'scores_voice': scores_voice, 'accuracy_voice':get_accuracy(scores_voice),
            'scores_balance': scores_balance, 'accuracy_balance':get_accuracy(scores_balance),
            'scores_gait': scores_gait, 'accuracy_gait':get_accuracy(scores_gait),
            'scores_tap': scores_tap, 'accuracy_tap':get_accuracy(scores_tap),
            'scores_react': scores_react, 'accuracy_react':get_accuracy(scores_react)}



In [ ]:
# combine select weights and lambda together 
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, RobustScaler

kf = KFold(n_splits=10, shuffle=True)

fold_accuracy = []
fold_weights = []
fold_train_pt = []
fold_test_pt = []
fold_scaler = []
fold_res_train = []
fold_res_test = []

index_cols = ['id', 'date', 'time', 'session', 'tsp', 'day_order']
for train, test in kf.split(pts):
    train_pt = pts[train]
    test_pt = pts[test]
    # train dataset
    dataset_train = dataset_before[dataset_before.id.isin(train_pt)].\
        append(dataset_after[dataset_after.id.isin(train_pt)])
    XTrain = dataset_train.drop(index_cols, axis=1)
    XTrainTsp = pd.to_datetime(dataset_train.tsp)
    XTrainTimeVector = (XTrainTsp - np.datetime64('1970-01-01 00:00:00')) / np.timedelta64(1, 's')
    XTrainTimeVector.head()
    scaler = RobustScaler()
    XTrainScaled = scaler.fit_transform(XTrain)
    train_ids = dataset_train[['id','tsp']].sort_values(by=['id', 'tsp']).id.values
    train_ids_idx = dataset_train[['id','tsp']].sort_values(by=['id', 'tsp']).index.values
    sequtial_idx_1 = train_ids[:-1] == train_ids[1:]
    sequtial_idx_2 = np.insert(sequtial_idx_1, 0, False)
    sequtial_idx_1 = np.append(sequtial_idx_1, False)
    sm_on_idx = []
    sm_off_idx = []


    for i, idx_1 in enumerate(train_ids_idx[sequtial_idx_1]):
        idx_2 = train_ids_idx[sequtial_idx_2][i]
        row_i_1 = dataset_train.index.get_loc(idx_1)
        row_i_2 = dataset_train.index.get_loc(idx_2)
        sm_on_idx.append(row_i_2)
        sm_off_idx.append(row_i_1)

    sm_on_idx = np.add(sm_on_idx,1)
    sm_off_idx = np.add(sm_off_idx,1)
    res = train_model_in_R()
    print "split:", train_pt, test_pt
    
    # test dataset
    dataset_test = dataset_before[dataset_before.id.isin(test_pt)]\
        .append(dataset_after[dataset_after.id.isin(test_pt)])
    XTest = dataset_test.drop(index_cols, axis=1)
    XTestScaled = scaler.transform(XTest)        

    acc = []
    weights = []
    res_train_list = []
    res_test_list = []
    for i,la in enumerate(lo):
        %R -i i -o w w = res[[i+1]]$estimate
        res_train = evaluate(XTrainScaled, w)
        res_test = evaluate(XTestScaled, w)
        acc.append([(i+1), lo[i], ls[i], res_train['accuracy'], res_test['accuracy']])
        weights.append(w)
        res_train_list.append(res_train)
        res_test_list.append(res_test)
    fold_accuracy.append(acc)
    fold_weights.append(weights)
    fold_train_pt.append(train_pt)
    fold_test_pt.append(test_pt)
    fold_scaler.append(scaler)
    fold_res_train.append(res_train_list)
    fold_res_test.append(res_test_list)
    

# Performance Evaluation

In [ ]:
# select best hyper parameters from validation set and apply to test set
fold_models = []
best_acc = []
for i, acc in enumerate(fold_accuracy):
    model = {}
    train_acc = [val[3] for val in acc]
    test_acc = [val[4] for val in acc]
    best_wi = train_acc.index(max(train_acc))
    model['weights'] = fold_weights[i][best_wi]
    model['scaler'] = fold_scaler[i]
    model['wi'] = best_wi
    fold_models.append(model)
    print best_wi, train_acc[best_wi], test_acc[best_wi], max(test_acc)
    best_acc.append(test_acc[best_wi])
np.mean(best_acc)

In [ ]:

def weight_analysis(models, names):
    for i, m in enumerate(models):
        if i == 0:
            weights = m['weights']
        else:
            weights += m['weights']
    weights /= len(models)
    df = pd.DataFrame(weights, index=names, columns=['weight'])
    ii = df.weight.abs().sort_values(ascending=False)
    df["scaled"] = df.weight / max(df.weight.abs())
    print df.scaled[ii.index].head(30)
    return df

def get_fold_test_scores(test_pt, test_scores):
    # get idx: id and tsp
    df = dataset_before[dataset_before.id.isin(test_pt)][['id','tsp']]\
        .append(dataset_after[dataset_after.id.isin(test_pt)][['id','tsp']])
    # get score columns
    for item in test_scores:
        if item.startswith('scores'):
            df[item] = test_scores[item]
    return df
    
    

for i, model in enumerate(fold_models):
    wi = model['wi']
    print i, wi
    if i == 0:
        scores = get_fold_test_scores(fold_test_pt[i], fold_res_test[i][wi])
    else:
        scores = scores.append(get_fold_test_scores(fold_test_pt[i], fold_res_test[i][wi]))

scores.tsp = pd.to_datetime(scores.tsp)

df = weight_analysis(fold_models, XTrain.columns.values)
print len(df)

In [ ]:
df["abs_w"] = df.weight.abs()
df["abs_w_scale"] = 100* (df.abs_w - df.abs_w.min()) / df.abs_w.max() - df.abs_w.min()
df.abs_w_scale.sort_values(ascending=False).to_csv('sorted_weight.1231.csv', index=True)

In [ ]:
df.abs_w_scale[df.abs_w_scale > 30].sort_values(ascending=False)

In [ ]:
df.abs_w_scale[df.abs_w_scale < 1].count()

In [ ]:
df.abs_w_scale[df.abs_w_scale < 1].sort_values(ascending=False)

In [ ]:
scores_before = pd.DataFrame()
for id in scores.id.unique():
    pt_scores = scores[scores.id == id]
    l = len(pt_scores)/2
    scores_before = scores_before.append(pt_scores[:l])
    
scores.to_csv('pt_scores_v2.1231.csv', index=False)
scores_before.to_csv('pt_scores_v2_before.1231.csv', index=False)

In [ ]:
scaled_scores = scores.copy()
scaler100 = RobustScaler(quantile_range=(15.0, 85.0))
scaler100.fit(scaled_scores.scores)
scaled_scores['scores_scaled'] = 50*scaler100.transform(scaled_scores.scores) + 50
scaled_scores.scores_scaled[(scaled_scores.scores_scaled > 100)] = 100
scaled_scores.scores_scaled[(scaled_scores.scores_scaled < 0)] = 0

print scaled_scores.scores_scaled.mean(), scaled_scores.scores_scaled.std() \
    , scaled_scores.scores_scaled.quantile(.25), scaled_scores.scores_scaled.quantile(.75)
scaled_scores.head()

In [ ]:
scaled_stats = scaled_scores.copy()

scaler100.scale_, scaler100.center_

c = 50*(-scaler100.center_/scaler100.scale_)/5
print(c)
scaled_stats['scores_voice'] = c + scaled_stats.scores_voice/scaler100.scale_*50
scaled_stats['scores_balance'] = c + scaled_stats.scores_balance/scaler100.scale_*50
scaled_stats['scores_tap'] = c + scaled_stats.scores_tap/scaler100.scale_*50
scaled_stats['scores_react'] = c + scaled_stats.scores_react/scaler100.scale_*50
scaled_stats['scores_gait'] = c + scaled_stats.scores_gait/scaler100.scale_*50
scaled_stats[['scores_voice', 'scores_balance', 'scores_gait', 'scores_tap', 'scores_react', 'scores_scaled']].abs().sum()/305694.031681